In [192]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
import sklearn


traindf = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
togeth = pd.concat([testdf,traindf])

In [193]:
togeth.isna().sum().sum()

7718

In [194]:
togeth

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,NaN
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,NaN
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,NaN
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,NaN
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


Need to set to categories after !!


In [195]:
togeth[togeth.PassengerId == '0293_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
135,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,0.0,0.0,Tauxon Suptibler,NaN


In [222]:
def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df.Group = df.Group.astype('float')
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    df.GroupNumber = df.GroupNumber.astype('float')
    return df

def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    custom_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    df['CabinDeck'] = pd.Categorical(df['CabinDeck'], categories=custom_order, ordered=True)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df

def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    return df

def fill_home_planet_by_last_name(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

def fill_home_planet_by_group(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['Group'] == row['Group']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

def home_planet_from_cabindeck_abc(df):
    df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df

def home_planet_from_cabindeck_g(df):
    df.loc[df['CabinDeck']== 'G', 'HomePlanet'] = df.loc[df['CabinDeck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df

def cabin_dm(df):
    df = split_group_number(df)
    df = split_cabin(df)
    df = split_names(df)
    df = df.sort_values(by = ['Group','GroupNumber'])
    df.reset_index(inplace = True)
    df.drop(columns = 'index',inplace = True)
    df = fill_home_planet_by_group(df)
    df = fill_home_planet_by_last_name(df)
    df = home_planet_from_cabindeck_abc(df)
    df = home_planet_from_cabindeck_g(df)
    return df


In [341]:
togeth = cabin_dm(togeth)

In [198]:
def indexfromid(id):
    return togeth[togeth.PassengerId == id].index.item()

def idfromindex(index):
    return togeth.loc[index].PassengerId
    
def cab_fill(cabfilllist):
    for item in cabfilllist:
        ind = indexfromid(item[0])
        togeth.at[ind,'Cabin'] = item[1]
    

In [199]:
temp = togeth.copy()

In [385]:
togeth = temp.copy()

In [440]:
togeth.at[indexfromid('3034_01'),'Cabin'] = 'B/98/P'
togeth.at[indexfromid('3053_01'),'Cabin'] = 'B/99/P'
togeth.at[indexfromid('4953_01'),'Cabin'] = 'C/193/S'
togeth.at[indexfromid('5480_01 '),'Cabin'] = 'C/214/S'

togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'



ValueError: can only convert an array of size 1 to a Python scalar

In [441]:
decks_by_homeplanet = {'Europa':list(togeth[togeth.HomePlanet == 'Europa'].CabinDeck.dropna().unique()),
    'Earth':list(togeth[togeth.HomePlanet == 'Earth'].CabinDeck.dropna().unique()),
    'Mars':list(togeth[togeth.HomePlanet == 'Mars'].CabinDeck.dropna().unique())}

def caboptionsbyplanet(planet,df):
    df = split_cabin(df)
    decks = decks_by_homeplanet[planet]
    sides = {'P':0,'S':0}
    caboptions = {}
    for i,row in df[df.HomePlanet == planet].iterrows():
        if pd.isna(row.Cabin):
            bef = df.iloc[:i].dropna(subset=['Cabin'])
            aft = df.iloc[i+1:].dropna(subset=['Cabin'])
            for deck in decks:
                for side in sides.keys():
                    before = bef[(bef.CabinDeck == deck) & (bef.CabinSide == side)]
                    after = aft[(aft.CabinDeck == deck) & (aft.CabinSide == side)]
                    if len(before) == 0:
                        minn = -2
                    else:
                        minn = max(list(before.CabinNum))
                    if len(after) == 0:
                        maxx = -2
                    else:
                        maxx = min(list(after.CabinNum))
                    if row.PassengerId in caboptions:
                        caboptions[row.PassengerId].append([minn,maxx,deck,side])
                    else:
                        caboptions[row.PassengerId] = [[minn,maxx,deck,side]]
    return caboptions, df

def one_option_solo_group(potential_spots,df):
    cabin_to_fill = []
    for passenger in potential_spots:
        if len(df[df.Group == int(passenger[:4])]) == 1:
            ind = -1
            for i,decks in enumerate(potential_spots[passenger]):
                if decks[1] - decks[0] != 1:
                    if ind == -1:
                        ind = i
                    elif decks[1] != 0:
                        ind = -2
            if ind != -2 and ind != -1:
                #check this line
                cabin_to_fill.append([passenger, potential_spots[passenger][ind][2] + "/" + str(potential_spots[passenger][ind][0] + 1) +"/" + potential_spots[passenger][ind][3]])
    return cabin_to_fill

def no_options(potential_spots,df):
        #no options
    cabin_to_fill = []
    for passenger in potential_spots:
        count = 0
        no_potential_spots = True
        for decks in potential_spots[passenger]:
            if (decks[1] - decks[0] > 1 and decks[1] != 0):
                no_potential_spots = False
        if no_potential_spots:
            if len(list(df[df.Group == int(passenger[:4])].dropna(subset = 'Cabin').Cabin.unique())) == 1:
                cabin_to_fill.append([passenger,list(df[df.Group == int(passenger[:4])].dropna(subset = 'Cabin').Cabin.unique())[0]])
    return cabin_to_fill
 
def only_one_thatll_fit(potential_spots,df):
    cabin_to_fill = []
    for i1,passenger in enumerate(potential_spots.keys()):            
        for i2, cabins in enumerate(potential_spots[passenger]):
            if cabins[1] != 0 and cabins[1] == cabins[0] + 2:
                good = True
                low = i1 - 1
                high = i1  + 1
                lowchecking = True
                highchecking = True
                while lowchecking and low >= 0:
                    option = potential_spots[list(potential_spots.keys())[low]]
                    for choices in option:
                        if choices[2] == cabins[2] and choices[3] == cabins[3]:
                            lowchecking = False
                            if choices[1] >= cabins[0] + 1:
                                good = False
                    low -=1
                while highchecking and good and high <= len(potential_spots.keys()) -1 :
                    option = potential_spots[list(potential_spots.keys())[high]]
                    for choices in option:
                        if choices[2] == cabins[2] and choices[3] == cabins[3]:
                            highchecking = False
                            if choices[0] <= cabins[1] - 1:
                                good = False
                    high += 1                                   
                if good:
                    cabin_to_fill.append([passenger, cabins[2] + "/" + str(cabins[0] + 1) +"/" + cabins[3]])
    return cabin_to_fill

def cab_filler(df):
    
    for planet in ['Europa','Mars','Earth']:
        planet_cab_options, df = caboptionsbyplanet(planet,df)
        cab_fill(no_options(planet_cab_options,df))
    
    
    for planet in ['Europa','Mars','Earth']:
        planet_cab_options, df = caboptionsbyplanet(planet,df)
        cab_fill(one_option_solo_group(planet_cab_options,df))


    europacaboptions = caboptionsbyplanet('Europa',df)[0]
    marscaboptions = caboptionsbyplanet('Mars',df)[0]
    earthcaboptions = caboptionsbyplanet('Earth',df)[0]

    indexlist = [indexfromid(x) for x in list(europacaboptions.keys())] + [indexfromid(x) for x in list(marscaboptions.keys())] + [indexfromid(x) for x in list(earthcaboptions.keys())]
    indexlist.sort()
    idlist = [idfromindex(x) for x in indexlist]
    global_cab_options = {}
    for id in idlist:
        if id in europacaboptions:
            global_cab_options[id] = europacaboptions[id]
        elif id in marscaboptions:
            global_cab_options[id] = marscaboptions[id]
        else:
            global_cab_options[id] = earthcaboptions[id]

    cab_fill(only_one_thatll_fit(global_cab_options,df))

    
    return global_cab_options


  
    


In [442]:
glob = cab_filler(togeth)

In [444]:
togeth.isna().sum()

PassengerId        0
HomePlanet        10
CryoSleep        310
Cabin             59
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Transported     4277
Group              0
GroupNumber        0
CabinDeck         59
CabinSide         59
CabinNum          59
FirstName        294
LastName         294
dtype: int64

In [409]:
europacaboptions = caboptionsbyplanet('Europa',togeth)[0]
marscaboptions = caboptionsbyplanet('Mars',togeth)[0]
earthcaboptions = caboptionsbyplanet('Earth',togeth)[0]

indexlist = [indexfromid(x) for x in list(europacaboptions.keys())] + [indexfromid(x) for x in list(marscaboptions.keys())] + [indexfromid(x) for x in list(earthcaboptions.keys())]
indexlist.sort()
idlist = [idfromindex(x) for x in indexlist]
global_cab_options = {}
for id in idlist:
    if id in europacaboptions:
        global_cab_options[id] = europacaboptions[id]
    elif id in marscaboptions:
        global_cab_options[id] = marscaboptions[id]
    else:
        global_cab_options[id] = earthcaboptions[id]

for item in global_cab_options:
    print(item,global_cab_options[item])

0293_01 [[12, 14, 'B', 'P'], [7, 8, 'B', 'S'], [2, 3, 'A', 'P'], [3, 4, 'A', 'S'], [6, 7, 'C', 'P'], [12, 14, 'C', 'S'], [10, 11, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [17, 18, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0310_01 [[12, 14, 'B', 'P'], [8, 9, 'B', 'S'], [3, 4, 'A', 'P'], [3, 4, 'A', 'S'], [7, 8, 'C', 'P'], [12, 14, 'C', 'S'], [11, 12, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [19, 20, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0348_02 [[76, 77, 'F', 'P'], [64, 65, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [21, 22, 'E', 'S']]
0364_02 [[81, 82, 'F', 'P'], [68, 69, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [22, 23, 'E', 'S']]
0374_02 [[86, 87, 'F', 'P'], [68, 69, 'F', 'S'], [62, 63, 'G', 'P'], [53, 54, 'G', 'S'], [19, 22, 'E', 'P'], [22, 23, 'E', 'S']]
0992_04 [[37, 38, 'B', 'P'], [40, 41, 'B', 'S'], [7, 8, 'A', 'P'], [10, 11, 'A', 'S'], [35, 36, 'C', 'P'], [38, 39, 'C', 'S'], [38, 39,

In [439]:
for item in glob:
    if item[-1] == '1':
        if len(togeth[togeth.Group == int(item[:-3])]) == 1:
            print(item,glob[item])

0293_01 [[12, 14, 'B', 'P'], [7, 8, 'B', 'S'], [2, 3, 'A', 'P'], [3, 4, 'A', 'S'], [6, 7, 'C', 'P'], [12, 14, 'C', 'S'], [10, 11, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [17, 18, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0310_01 [[12, 14, 'B', 'P'], [8, 9, 'B', 'S'], [3, 4, 'A', 'P'], [3, 4, 'A', 'S'], [7, 8, 'C', 'P'], [12, 14, 'C', 'S'], [11, 12, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [19, 20, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
1041_01 [[37, 38, 'B', 'P'], [40, 41, 'B', 'S'], [7, 8, 'A', 'P'], [12, 13, 'A', 'S'], [37, 38, 'C', 'P'], [39, 41, 'C', 'S'], [39, 40, 'D', 'P'], [35, 37, 'D', 'S'], [57, 59, 'E', 'P'], [74, 75, 'E', 'S'], [-2, 0, 'T', 'P'], [0, 1, 'T', 'S']]
1095_01 [[38, 39, 'B', 'P'], [42, 43, 'B', 'S'], [8, 9, 'A', 'P'], [12, 13, 'A', 'S'], [41, 42, 'C', 'P'], [39, 41, 'C', 'S'], [42, 43, 'D', 'P'], [35, 37, 'D', 'S'], [61, 62, 'E', 'P'], [75, 76, 'E', 'S'], [0, 1, 'T', 'P'], [0, 1, 'T', 'S']]
2513_01 [[518, 520, 'F', 'P'], [483

In [413]:
glob.keys()

dict_keys(['0293_01', '0310_01', '0348_02', '0364_02', '0374_02', '0992_04', '1006_03', '1011_01', '1041_01', '1095_01', '1688_02', '1709_03', '2092_03', '2425_05', '2513_01', '2514_01', '3287_02', '3411_02', '3598_01', '3599_01', '3712_01', '4005_04', '4953_01', '4974_02', '5098_02', '5480_01', '5601_03', '5642_02', '6028_04', '6048_01', '6060_01', '6302_04', '6363_01', '6405_02', '6612_03', '6612_05', '7182_01', '7183_01', '7353_03', '7368_01', '7429_01', '7440_01', '7442_02', '7463_01', '7469_01', '7556_03', '7890_03', '7942_02', '7948_01', '7983_01', '7995_01', '8129_01', '8728_07', '9057_01', '9062_02', '9069_03', '9070_01', '9081_03', '9223_01', '9223_02', '9238_05'])

In [426]:
togeth[togeth.Group == 9057]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
12651,9057_01,Europa,False,NaN,55 Cancri e,36.0,True,132.0,3479.0,0.0,3786.0,0.0,Coxan Statch,False,9057.0,1.0,NaN,<NA>,<NA>,Coxan,Statch
12652,9057_02,Europa,True,B/296/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Sitron Statch,True,9057.0,2.0,B,P,296,Sitron,Statch


In [421]:
togeth[togeth.Cabin == 'G/1476/P']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
12692,9081_01,Earth,False,G/1476/P,TRAPPIST-1e,8.0,False,0.0,0.0,0.0,0.0,0.0,Hanna Clemondsey,False,9081.0,1.0,G,P,1476,Hanna,Clemondsey
12695,9081_04,Earth,False,G/1476/P,TRAPPIST-1e,9.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True,9081.0,4.0,G,P,1476,<NA>,<NA>
12698,9081_07,Earth,True,G/1476/P,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Aliey Clemondsey,True,9081.0,7.0,G,P,1476,Aliey,Clemondsey
12699,9081_08,Earth,True,G/1476/P,PSO J318.5-22,3.0,False,0.0,0.0,0.0,0.0,0.0,Virgie Clemondsey,True,9081.0,8.0,G,P,1476,Virgie,Clemondsey


In [423]:
togeth[togeth.PassengerId== '4637_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
6493,4637_01,Mars,False,E/300/S,TRAPPIST-1e,24.0,False,672.0,0.0,501.0,0.0,NaN,Tark Ches,False,4637.0,1.0,E,S,300,Tark,Ches


In [347]:
togeth

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,1.0,1.0,B,P,0,Maham,Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,2.0,1.0,F,S,0,Juanna,Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,3.0,1.0,A,S,0,Altark,Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,3.0,2.0,A,S,0,Solam,Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,4.0,1.0,F,S,1,Willy,Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9277_01,Earth,True,G/1498/S,PSO J318.5-22,43.0,False,0.0,0.0,0.0,0.0,0.0,Lilace Leonzaley,NaN,9277.0,1.0,G,S,1498,Lilace,Leonzaley
12966,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,9278.0,1.0,G,S,1499,Kurta,Mondalley
12967,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,9279.0,1.0,G,S,1500,Fayey,Connon
12968,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,9280.0,1.0,E,S,608,Celeon,Hontichre


In [361]:
togeth[togeth.PassengerId == '3053_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
4254,3053_01,Europa,False,B/98/P,55 Cancri e,52.0,False,0.0,2.0,0.0,10976.0,810.0,Ainor Fuelisent,False,3053.0,1.0,B,P,98,Ainor,Fuelisent


In [355]:
togeth[togeth.Cabin.isna()].tail(50)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
2442,1709_03,Mars,False,NaN,TRAPPIST-1e,35.0,False,1313.0,0.0,24.0,0.0,1.0,Bleark Minen,False,1709.0,3.0,NaN,<NA>,<NA>,Bleark,Minen
2970,2092_03,Mars,False,NaN,TRAPPIST-1e,13.0,NaN,6726.0,0.0,1605.0,1266.0,0.0,Cray Stpie,False,2092.0,3.0,NaN,<NA>,<NA>,Cray,Stpie
3416,2425_05,Earth,True,NaN,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Coley Tranklinay,NaN,2425.0,5.0,NaN,<NA>,<NA>,Coley,Tranklinay
3529,2513_01,Earth,False,NaN,TRAPPIST-1e,28.0,False,0.0,55.0,0.0,656.0,0.0,Loree Mathison,False,2513.0,1.0,NaN,<NA>,<NA>,Loree,Mathison
3530,2514_01,Earth,False,NaN,TRAPPIST-1e,36.0,False,3.0,755.0,0.0,33.0,0.0,Ianne Doughan,NaN,2514.0,1.0,NaN,<NA>,<NA>,Ianne,Doughan
4569,3287_02,Earth,False,NaN,TRAPPIST-1e,22.0,False,718.0,2.0,50.0,0.0,0.0,Jerrye Carlsonney,True,3287.0,2.0,NaN,<NA>,<NA>,Jerrye,Carlsonney
4751,3411_02,Mars,False,NaN,TRAPPIST-1e,22.0,False,1127.0,0.0,2451.0,96.0,0.0,Naish Harta,NaN,3411.0,2.0,NaN,<NA>,<NA>,Naish,Harta
5016,3598_01,Earth,False,NaN,55 Cancri e,19.0,False,616.0,0.0,46.0,0.0,14.0,Dony Olivasquez,False,3598.0,1.0,NaN,<NA>,<NA>,Dony,Olivasquez
5017,3599_01,Earth,False,NaN,TRAPPIST-1e,46.0,False,48.0,856.0,0.0,0.0,4.0,Philda Newtontoss,NaN,3599.0,1.0,NaN,<NA>,<NA>,Philda,Newtontoss
5193,3712_01,Earth,True,NaN,55 Cancri e,0.0,False,0.0,0.0,0.0,0.0,0.0,Diandi Blacks,NaN,3712.0,1.0,NaN,<NA>,<NA>,Diandi,Blacks


In [349]:
togeth.isna().sum()

PassengerId        0
HomePlanet        10
CryoSleep        310
Cabin             61
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Transported     4277
Group              0
GroupNumber        0
CabinDeck         61
CabinSide         61
CabinNum          61
FirstName        294
LastName         294
dtype: int64

In [350]:
togeth[togeth.Cabin == 'A/7/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
1001,0709_01,Europa,False,A/7/S,55 Cancri e,25.0,False,0.0,4949.0,0.0,452.0,15.0,Astorux Missefle,True,709.0,1.0,A,S,7,Astorux,Missefle
1002,0709_02,Europa,True,A/7/S,55 Cancri e,25.0,NaN,0.0,NaN,0.0,0.0,0.0,Thaban Missefle,True,709.0,2.0,A,S,7,Thaban,Missefle
1003,0709_03,Europa,True,A/7/S,55 Cancri e,23.0,False,0.0,0.0,NaN,0.0,0.0,Props Missefle,True,709.0,3.0,A,S,7,Props,Missefle


In [362]:
def print_rows_with_different_group(df):
    unique_cabins = df['Cabin'].unique()
    
    for cabin in unique_cabins:
        group_list = df[df['Cabin'] == cabin]['Group'].unique()
        
        if len(group_list) > 1:
            print(cabin)

# Call the function with your DataFrame
print_rows_with_different_group(togeth)


B/98/P
